In [ ]:
!pip install gradio git+https://github.com/huggingface/transformers torch soundfile librosa pytube

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-dkrrhu0g
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-dkrrhu0g
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 56 kB 2.6 MB/s 


Using Processor + Batch Decode

Cuts the audio off after 30 seconds

In [ ]:
import pytube as pt
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
import soundfile

yt = pt.YouTube("https://www.youtube.com/watch?v=dd1kN_myNDs")
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

MODEL_NAME = "openai/whisper-small.en"

device = "cuda" if torch.cuda.is_available() else "cpu"

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

def load_and_fix_data(input_file):  
    speech, sample_rate = librosa.load(input_file)
    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]
    if sample_rate !=16000:
        speech = librosa.resample(speech, sample_rate,16000)
    speech = librosa.to_mono(speech)
    return speech


def transcribe(Microphone, File_Upload=None):
    warn_output = ""
    if (Microphone is not None) and (File_Upload is not None):
        warn_output = "WARNING: You've uploaded an audio file and used the microphone. " \
                      "The recorded file from the microphone will be used and the uploaded audio will be discarded.\n"
        file = Microphone

    elif (Microphone is None) and (File_Upload is None):
        return "ERROR: You have to either use the microphone or upload an audio file"

    elif Microphone is not None:
        file = Microphone
    else:
        file = File_Upload
    
    speech_data = load_and_fix_data(file)

    inputs = processor(speech_data, return_tensors="pt", sampling_rate=16_000).input_features.to(device)
#    forced_decoder_ids = processor.get_decoder_prompt_ids(language=lang, task="transcribe")

    predicted_ids = model.generate(inputs, max_length=480)
    text = processor.batch_decode(predicted_ids, skip_special_tokens=True, normalize=True)[0]    

    return warn_output + text

transcribe("audio.mp3")

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


'dear fellow scholars this is 2 minute papers with karoly zsolnai feher there are many ai techniques that are able to look at a still image and identify objects textures human poses and object parts in them really well however in the age of the internet we have videos everywhere so an important question would be how we could do the same for these animations'

Using Pipeline

Chunks quite alright!

In [ ]:
import pytube as pt
import torch
from transformers import pipeline
import librosa
import soundfile

yt = pt.YouTube("https://www.youtube.com/watch?v=dd1kN_myNDs")
stream = yt.streams.filter(only_audio=True)[0]
stream.download(filename="audio.mp3")

MODEL_NAME = "openai/whisper-small.en"

device = "cuda" if torch.cuda.is_available() else "cpu"

pipe = pipeline(task="automatic-speech-recognition", model=MODEL_NAME, chunk_length_s=30)

def load_and_fix_data(input_file):  
    speech, sample_rate = librosa.load(input_file)
    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]
    if sample_rate !=16000:
        speech = librosa.resample(speech, sample_rate,16000)
    speech = librosa.to_mono(speech)
    return speech


def transcribe(Microphone, File_Upload=None):
    warn_output = ""
    if (Microphone is not None) and (File_Upload is not None):
        warn_output = "WARNING: You've uploaded an audio file and used the microphone. " \
                      "The recorded file from the microphone will be used and the uploaded audio will be discarded.\n"
        file = Microphone

    elif (Microphone is None) and (File_Upload is None):
        return "ERROR: You have to either use the microphone or upload an audio file"

    elif Microphone is not None:
        file = Microphone
    else:
        file = File_Upload
    
    speech_data = load_and_fix_data(file)
    text = pipe(speech_data) 

    return warn_output + text["text"]

transcribe("audio.mp3")

/usr/local/lib/python3.8/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
Using `chunk_length_s` is very experimental with seq2seq models. The results will not necessarily be entirely accurate and will have caveats. More information: https://github.com/huggingface/transformers/pull/20104. Ignore this warning with pipeline(..., ignore_warning=True)
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 448 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


" Dear Fellow Scholars, this is Two Minute Papers with Károly Zsolnai-Fehér. There are many AI techniques that are able to look at a still image and identify objects, textures, human poses, and object parts in them really well. However, in the age of the Internet, we have videos everywhere, so an important question would be how we could do the same for these animations. One of the key ideas in this paper is that the frames of these videos are not completely independent, and they share a lot of information, so after we make our initial predictions on what is where exactly, these predictions from the previous frame can almost always be reused with a little modification. Not only that, but here you can see with these results that it can also deal with momentary occlusions and is ready to track objects that rotate or time. A key part of this method is that one, it looks back and forth in these videos to update these labels, and second, it learns in a self-supervised manner, which means tha

In [ ]:
import gradio as gr
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
import soundfile

MODEL_NAME = "openai/whisper-small"
lang = "ja"

device = "cuda" if torch.cuda.is_available() else "cpu"

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)


def load_and_fix_data(input_file):  
    speech, sample_rate = librosa.load(input_file)
    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]
    if sample_rate !=16000:
        speech = librosa.resample(speech, sample_rate,16000)
    speech = librosa.to_mono(speech)
    return speech


def transcribe(Microphone, File_Upload):
    warn_output = ""
    if (Microphone is not None) and (File_Upload is not None):
        warn_output = "WARNING: You've uploaded an audio file and used the microphone. " \
                      "The recorded file from the microphone will be used and the uploaded audio will be discarded.\n"
        file = Microphone

    elif (Microphone is None) and (File_Upload is None):
        return "ERROR: You have to either use the microphone or upload an audio file"

    elif Microphone is not None:
        file = Microphone
    else:
        file = File_Upload
    
    speech_data = load_and_fix_data(file)

    inputs = processor(speech_data, return_tensors="pt", sampling_rate=16_000).input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language=lang, task="transcribe")

    predicted_ids = model.generate(inputs, max_length=480_000, forced_decoder_ids=forced_decoder_ids)
    text = processor.batch_decode(predicted_ids, skip_special_tokens=True, normalize=True)[0]    

    return warn_output + text


iface = gr.Interface(
    fn=transcribe,
    inputs=[
        gr.inputs.Audio(source="microphone", type='filepath', optional=True),
        gr.inputs.Audio(source="upload", type='filepath', optional=True),
    ],
    outputs="text",
    layout="horizontal",
    theme="huggingface",
    title="[WFTE] Whisper model showcase",
    description="Demo for showcasing fine-tuned OpenAI Whisper models from WFTE.",
    allow_flagging='never',
)

iface.launch(enable_queue=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:319: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your components from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `layout` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/interface.py:332: UserWarning: Currently, only the 'default' theme is supported.
  warnings.warn("Currently, only the 'default' theme is supported.")


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://aab2b77e7afe7f5d.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


In [ ]:
import torch
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import librosa
import soundfile

SAMPLE_RATE = 16000
MODEL_NAME = "openai/whisper-small"
lang = "ja"

device = "cuda" if torch.cuda.is_available() else "cpu"

model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME).to(device)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

def load_and_fix_data(input_file):  
    speech, sample_rate = librosa.load(input_file)
    if len(speech.shape) > 1: 
        speech = speech[:,0] + speech[:,1]
    if sample_rate !=16000:
        speech = librosa.resample(speech, sample_rate,16000)
    speech = librosa.to_mono(speech)
    return speech


def transcribe(Microphone, File_Upload=None):
    warn_output = ""
    if (Microphone is not None) and (File_Upload is not None):
        warn_output = "WARNING: You've uploaded an audio file and used the microphone. " \
                      "The recorded file from the microphone will be used and the uploaded audio will be discarded.\n"
        file = Microphone

    elif (Microphone is None) and (File_Upload is None):
        return "ERROR: You have to either use the microphone or upload an audio file"

    elif Microphone is not None:
        file = Microphone
    else:
        file = File_Upload
    
    speech_data = load_and_fix_data(file)

    inputs = processor(speech_data, return_tensors="pt", sampling_rate=16000).input_features.to(device)
    forced_decoder_ids = processor.get_decoder_prompt_ids(language=lang, task="transcribe")

    predicted_ids = model.generate(inputs, max_length=480_000, forced_decoder_ids=forced_decoder_ids)
    text = processor.batch_decode(predicted_ids, skip_special_tokens=True, normalize=True)[0]    

    return warn_output + text

In [ ]:
transcribe("result.flac")

It is strongly recommended to pass the `sampling_rate` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


'キムラさんに電話をかしてもらいました'